In [1]:
from sklearn.base import TransformerMixin
import nltk  
from nltk import word_tokenize

In [2]:
class NLTKBOW(TransformerMixin):
    def fit(self ,X , y = None):
        return self
    def transform(self, X):
        return [{ word : True for word in word_tokenize(document) } for document in X ]


In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import BernoulliNB

In [4]:
import os
import json
input_filename  = os.path.join(os.path.expanduser("~"),"For Data Analysis","Data_Mining","Chapter06","Dataset","python_tweets.json")
label_filename = os.path.join(os.path.expanduser("~"),"For Data Analysis","Data_Mining","Chapter06","Dataset","python_classes.json")


In [5]:
tweets = []
with open(input_filename) as inf:
    for line in inf:
        if len(line.strip()) == 0:
            continue
        tweets.append(json.loads(line)['text'])

In [6]:
with open(label_filename) as inf:
    labels = json.load(inf)

In [62]:
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import cross_val_score
'''
此处流水线的作用是先将tweets数据集转化为一个词袋（出现的词则标记为TRUE）,后将词袋转化为可被分类器处理的矩阵(每行为不同文本，每列为不同的单词),
最后将数据集使用二元伯努利概型预测正确性

'''
pipeline = Pipeline([("split_words", NLTKBOW()),("vectorize", DictVectorizer() ),("naive_bayes", BernoulliNB() )])
scores = cross_val_score(pipeline , tweets , labels, scoring = 'f1')#默认会返回类别为1的f1值

In [63]:
import numpy as np
np.mean(scores)*100

53.60735171261487

In [83]:
#根据自己理解模拟pipeline过程， 但结果不正确，pipeline过程和estimator仍需理解原理
NLTKBOW().fit_transform(tweets,None)
vec = DictVectorizer()
vec.fit_transform(NLTKBOW().fit_transform(tweets,None)).toarray()
nb = BernoulliNB()
scores = cross_val_score(nb ,vec.fit_transform(NLTKBOW().fit_transform(tweets,None)).toarray(),labels,scoring = 'f1')
np.mean(scores)*100
#print(vec.fit_transform(NLTKBOW().fit_transform(tweets,None)).toarray().shape)


52.33333333333333

In [85]:
'''
从模型中获取前k个有用的特征
'''
model = pipeline.fit(tweets , labels)
nb = model.named_steps['naive_bayes']

In [104]:
feature_probabilities = nb.feature_log_prob_#将特征出现的概率对数化（由乘法转化为加法，防止乘0）
'''
x = np.array([3, 1, 2])
np.argsort(x)#返回a[1]<a[2]<a[0]即，1<2<3
>>>array([1, 2, 0], dtype=int64)

'''
top_feature = np.argsort(-feature_probabilities[1])[:50]#在出现概率从大到小的特征值中找到前50个特征值（输出为对应的数组下标）
dv = model.named_steps['vectorize']
for i , index_featrue in enumerate(top_feature):
    print(i , dv.feature_names_[index_featrue] ,np.exp(feature_probabilities[1][index_featrue]))

0 : 0.7962962962962962
1 @ 0.685185185185185
2 https 0.574074074074074
3 RT 0.5555555555555556
4 Python 0.4259259259259259
5 # 0.40740740740740744
6 . 0.35185185185185175
7 , 0.2592592592592592
8 in 0.2592592592592592
9 to 0.2407407407407407
10 Developer 0.2037037037037037
11 nodejs 0.2037037037037037
12 js 0.2037037037037037
13 with 0.2037037037037037
14 ( 0.16666666666666669
15 the 0.16666666666666669
16 ! 0.16666666666666669
17 ) 0.1481481481481481
18 a 0.1481481481481481
19 I 0.12962962962962962
20 node 0.12962962962962962
21 BoostlogHQ 0.12962962962962962
22 angularjs 0.12962962962962962
23 coding 0.12962962962962962
24 of 0.12962962962962962
25 and 0.11111111111111109
26 you 0.11111111111111109
27 is 0.11111111111111109
28 Facebook 0.09259259259259259
29 for 0.09259259259259259
30 10 0.09259259259259259
31 Top 0.09259259259259259
32 100DaysOfCode 0.09259259259259259
33 Javascript 0.09259259259259259
34 Make 0.09259259259259259
35 webdev 0.09259259259259259
36 projects 0.092592592

array([1, 2, 0], dtype=int64)